# Collect Satellite images of the farmers location from multiple timestamps

In [1]:
from python_scripts.connector import *
from python_scripts.helper import *

from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Succefully connected!


## Step 1: Get the necessary timestamps

In [2]:
def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots
    

In [3]:
slots = get_time_slots('2017-01-01', '2022-01-01', 5)
print("Yearly time windows")
for slot in slots:
    print(slot)

Yearly time windows
('2017-01-01', '2018-01-01')
('2018-01-01', '2019-01-01')
('2019-01-01', '2020-01-01')
('2020-01-01', '2020-12-31')


## Step 2: Get the necessary coordinates 

In [4]:
# convert coordinations_file (csv file) into a dataframe
coordinations_df = pd.read_excel('data/locations/FarmerLocationExtract4Interns_sentinel-hub.xlsx')

In [5]:
# Based on how far wail is with the conversion of the coordinates of the farmers location, we need to check whether or 
# the coordination needs to mdofied. 
coordinations_df

,Unnamed: 0,Name,Mobile (anonymized),Mouza,GP,Block,Land_ID1,Land_Size2,P1_Latitude,P1_Longitude
0,0,TEMP_NAME,PHONE01,Baksa,11 No Arrah,Nayagram,40,0.167,21.5439876,87.118996
1,1,TEMP_NAME,PHONE02,Banskhali,11 No Arrah,Nayagram,100/53,0.333,21.535550432,87.454971
2,2,TEMP_NAME,PHONE03,Pathrasol,12 No Chandrarekha,Nayagram,51,0.333,21.581197383,87.157488
3,3,TEMP_NAME,PHONE04,Ramkrishnapur,11 No Arrah,Nayagram,36/947,0.167,21.535747928,87.139477
4,4,TEMP_NAME,PHONE05,Bamdiha,11 No Arrah,Nayagram,92,0.333,21.55262056,87.229918
...,...,...,...,...,...,...,...,...,...,...
244,245,TEMP_NAME,PHONE246,Bamdiha,11 No Arrah,Nayagram,230,0.260,21.5547784,87.216591
245,246,TEMP_NAME,PHONE247,Bamdiha,11 No Arrah,Nayagram,47/1,0.260,21.55475356,87.216591
246,247,TEMP_NAME,PHONE248,Bamdiha,11 No Arrah,Nayagram,285,0.520,21.52133968,87.530772
247,248,TEMP_NAME,PHONE249,Bamdiha,11 No Arrah,Nayagram,51,0.520,21.55487128,87.215806


In [6]:
coordinations_df.columns

Index(['Unnamed: 0', 'Name', 'Mobile (anonymized)', 'Mouza', 'GP', 'Block',
       'Land_ID1', 'Land_Size2', 'P1_Latitude', 'P1_Longitude'],
      dtype='object')

In [7]:
def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['P1_Longitude'], x['P1_Latitude'], CRS.WGS84)
    coords = [lng - 0.1, lat - 0.1, lng + 0.1, lat + 0.1]
    return coords

In [8]:
geolocations  = coordinations_df[['P1_Longitude', 'P1_Latitude']].copy()

In [9]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

,P1_Longitude,P1_Latitude
14,87.242824,"21.54""95616"
15,87.234883,"21.54""8830"
16,87.234738,"21.54""329472"


In [10]:
geolocations['P1_Latitude'] = geolocations['P1_Latitude'].str.replace('"', '')

In [11]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

,P1_Longitude,P1_Latitude


In [12]:
geolocations = geolocations.astype("float64")

In [13]:
geolocations.dtypes

P1_Longitude    float64
P1_Latitude     float64
dtype: object

In [14]:
geolocations.iloc[:3]

,P1_Longitude,P1_Latitude
0,87.118996,21.543988
1,87.454971,21.535550
2,87.157488,21.581197


In [15]:
# geolocations.to_csv('coordinaten.csv', sep='\t')

## Download all the satellite images 

In [16]:
import time # time the performance
import os, shutil

In [17]:
def least_clouds():
    return { 
    "dataFilter": { 
        "maxCloudCoverage": 1
        } 
    } 

In [18]:
def empty_folder(folder):
    """
    delete files in folder 
    """
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    

In [19]:
empty_folder('../data/images')
empty_folder('../data/mask')
empty_folder('../data/ndvi_values_mask')

In [20]:
def get_request_(evalscript, time_interval, coords, data_folder):
    return SentinelHubRequest(
        data_folder=data_folder, 
        evalscript=evalscript, 
        input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A, 
            time_interval=time_interval,
            mosaicking_order='leastCC')], 
        bbox=BBox(coords,crs=CRS.WGS84), 
        config=config, 
        size=[512, 512], 
        responses=[SentinelHubRequest.output_response('default', MimeType.PNG)])
    
    

In [21]:
from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_forest import *
from eval_scripts.evalscript_ndvi import *

In [22]:
def download_images(geolocations, evalscript, slots, data_folder):
    for index, row in geolocations.iterrows(): # iterate through each location
        # create a list of requests
        coords = bbox_converter(row) # convert each location to a bbox

        list_of_requests = [get_request_(evalscript,slot, coords,data_folder) for slot in slots]
        list_of_requests = [request.download_list[0] for request in list_of_requests]
        
        # download data with multiple threads
        SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=8)

In [ ]:
start = time.perf_counter()
download_images(geolocations[:10],evalscript_true_color, slots, '../data/images' )
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

In [ ]:
#dowload mask - duurt heel lang
# start = time.perf_counter()
# download_images(geolocations.iloc[:50],evalscript_ndvi, slots, '../data/mask' )
# end = time.perf_counter()
# print("Done in: {} seconds".format( end - start))

In [ ]:
def download_mask_v2(geolocations, slots, data_folder):
    for index, rows in geolocations 
         coords = bbox_converter(row) # convert each location to a bbox
        for slot in slots:
            img_ndvi = sentinel_request(evalscript_ndvi_values, bbox, slot config, False, other_args=no_clouds)
            min_ndvi = 0.75
            ndvi_copy = img_ndvi.copy()
            labels = np.where(ndvi_copy > min_ndvi, 255, 0)
            img = Image.fromarray(labels)
            filename = "{}_{}_{}_{}_{}_{}".format(coords[0], coords[1], coords[2],coords[3], slot[0], slot[1])
            img.save(f'{data_folder}/{filename}.png', 'PNG')

In [ ]:
start = time.perf_counter()
download_mask_v2(geolocations[:10], slots,'../data/ndvi_values_mask' )
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

## Get and rename images

In [ ]:
import os
import json
import shutil

In [ ]:
def parse_json(path):
    input_json = open(path)
    pased_json = json.load(input_json)
    return pased_json

In [ ]:
def build_filename(json):

    item =  json['payload']['input']
    bbox = item['bounds']['bbox']
    date_start= item['data'][0]['dataFilter']['timeRange']['from'].split("T")[0]
    date_end  = item['data'][0]['dataFilter']['timeRange']['to'].split("T")[0]
    
    return "{}_{}_{}_{}_{}_{}".format(bbox[0], bbox[1], bbox[2],bbox[3], date_start, date_end) # TODO : check format and . in naamgeving 

In [ ]:
build_filename(parse_json(r'C:\Users\Amaryllis Lee\FARM-deforestation\data\images\1484b4269e568648b673959b5a955ce4\request.json'))

In [ ]:
def images_structure(rootdir,new_rootdir):
    for root, subdirectories, files in os.walk(rootdir):
        for subdirectory in subdirectories: # for each folder in dir:
            request_json = f'{rootdir}/{subdirectory}/request.json'
            response_png = f'{rootdir}/{subdirectory}/response.png'
            if os.path.isfile(request_json) and os.path.isfile(response_png):
                json_file = parse_json(request_json)
                new_filename = f'{new_rootdir}/{build_filename(json_file)}.png'
                shutil.copyfile(response_png,new_filename)
            # get image:
            #rename image
            #move to data amsk aand remove folder/subdir
            

In [ ]:
rootdir_true_colors = '../data/images'
new_rootdir_true_colors = '../data/true_colours_images_testset'
images_structure(rootdir_true_colors, new_rootdir_true_colors)

In [ ]:
# Modifying filename move mask to mask_png_testset
# rootdir_mask = '../data/mask'
# new_rootdir_mask= '../data/mask_png_testset'
# images_structure(rootdir, new_rootdir)